In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
data_LA = pd.read_csv("input/LA_total.csv",index_col=0)

data_LA

In [ ]:
data_XA = pd.read_csv("input/XA_total.csv",index_col=0)
data_XA

In [ ]:
data_LA.columns

In [ ]:
data_LA.info()

In [ ]:
data_LA_ = pd.DataFrame()
columns_LA = data_LA.columns
for col in columns_LA:
    try:
        df = data_LA[col].astype(np.float64)
        data_LA_ = pd.concat([data_LA_,df],axis=1)
    except:
        pass
    continue
data_LA_

In [ ]:
data_XA_ = pd.DataFrame()
columns_XA = data_XA.columns
for col in columns_XA:
    try:
        df = data_XA[col].astype(np.float64)
        data_XA_ = pd.concat([data_XA_,df],axis=1)
    except:
        pass
    continue
data_XA_

In [ ]:
# T檢驗

In [ ]:
from scipy.stats import ttest_ind, levene

In [ ]:
index = []
for col in data_LA_.columns:
    if ttest_ind(data_LA_[col],data_XA_[col])[1] < 0.05:
        index.append(col)

In [ ]:
index

In [ ]:
len(index)

In [ ]:
# 方差齐性
index_ = []
for col in data_LA_.columns:
    if levene(data_LA_[col],data_XA_[col])[1] > 0.05:
        if ttest_ind(data_LA_[col],data_XA_[col])[1] < 0.05:
            index_.append(col)
    else:
        if ttest_ind(data_LA_[col],data_XA_[col],equal_var=False)[1] < 0.05:
            index_.append(col)
index_

In [ ]:
len(index_)

In [ ]:
data_L_T = data_LA_[index_]
data_L_T

In [ ]:
data_X_T = data_XA_[index_]
data_X_T

In [ ]:
# data = pd.concat([data_LA_,data_XA_])
data = pd.concat([data_L_T,data_X_T])
data = shuffle(data)
print(data.shape)
data

In [ ]:
target = data.iloc[:, 0]
target

In [ ]:
data.info()

In [ ]:
features = data.iloc[:,1:]
features

In [ ]:
features_bk = features.copy()
features_bk.info()

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
transfer = StandardScaler()
features_SS = transfer.fit_transform(features_bk)
features_SS
# 数据标准化很有必要

In [ ]:
type(features_SS)

In [ ]:
alphas_ = np.logspace(-2,0,300)
alphas_

In [ ]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(alphas = alphas_,cv = 10,max_iter = 100000).fit(features_SS,target)
lassocv.alpha_

In [ ]:
lassocv.coef_

In [ ]:
a = lassocv.coef_!=0
cnt = 0
for b in a:
    if b:
        cnt+=1
cnt

In [ ]:
features.columns[lassocv.coef_!=0]

In [ ]:
features = data[features.columns[lassocv.coef_!=0]]

In [ ]:
features

In [ ]:
#構建模型

In [ ]:
transfer = StandardScaler()
features_SS = transfer.fit_transform(features)
print(features_SS.shape)
features_SS


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_features, test_features, train_target, test_target = train_test_split(
    features_SS, target, 
    test_size = 0.15)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.linear_model import LogisticRegression

In [ ]:
# log = LogisticRegression(penalty='l1',max_iter = 10000000,solver='saga',C=0.001,n_jobs=-1)
log = LogisticRegression(penalty='l2',max_iter = 10000000,n_jobs=-1)

In [ ]:
log.fit(train_features, train_target)

In [ ]:
test_predict = log.predict(test_features)
test_predict

In [ ]:
test_target

In [ ]:
test_proba = log.predict_proba(test_features)
test_proba

In [ ]:
log.score(test_features,test_target)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve,accuracy_score

In [ ]:
accuracy_score(test_target,test_predict)

In [ ]:
confusion_matrix(test_target,test_predict)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(test_target,test_predict),
           xticklabels=['B','M'],
           yticklabels=['B','M'])

In [ ]:
y_score = log.decision_function(test_features)
y_score

In [ ]:
fpr, tpr, thresholds = roc_curve(test_target, y_score)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=3)

In [ ]:
train_features_ = poly_features.fit_transform(train_features)

In [ ]:
log_ = LogisticRegression(max_iter = 10000000)

In [ ]:
log_.fit(train_features_, train_target)

In [ ]:
test_features_ = poly_features.transform(test_features)
log_.score(test_features_,test_target)